In [ ]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
d_in = inputs.shape[1] #B

print(d_in)
d_out = 2 #C

3


In [ ]:
import torch.nn as nn
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        print(self.W_query)
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

Linear(in_features=3, out_features=2, bias=False)
tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [ ]:
queries = sa_v2.W_query(inputs) #A
keys = sa_v2.W_key(inputs)
# print(keys)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [ ]:
context_length=6
torch.ones(context_length, context_length)

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

When negative
infinity values (-∞) are present in a row, the softmax function treats them as zero
probability.

In [ ]:
mask= torch.triu(torch.ones(context_length,context_length),diagonal= 1) # exclude diagonal value for that diagoanl= 1
masked= attn_scores.masked_fill(mask.bool(),-torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [ ]:
atten_weight= torch.softmax(masked/ keys.shape[-1]**0.5 ,dim=1)
print(atten_weight)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [ ]:
torch.manual_seed(123)
dropout= torch.nn.Dropout(0.5)
ones= torch.ones(6,6)
exmple= dropout(ones)
print(exmple)

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


this type of dropout is implement on atten weight matrix as we put dropout=50% it dropout value averagly 50% from matrix<br>
In PyTorch nn.Module, register_buffer tells the model:

"mask" is not a learnable parameter (not updated by backpropagation).

But it moves with the model (goes to GPU with .cuda() or .to(device) and gets saved/loaded with state_dict).

3. self.W_key(x)

When you call it with input x, it performs:

keys= self.w_key(x) means keys= x*weight + b
So for each input token embedding (dimension = d_in), it creates a new vector (dimension = d_k) called the key.

in register_buffer (mask is variable with tensor which is upper tringular)


In [ ]:
class CausalAttention(nn.Module):
  def __init__(self, d_in,d_out,context_length,dropout,qkv_bias=False) :
      super().__init__()
      self.d_out= d_out
      self.w_query= nn.Linear(d_in,d_out,bias= qkv_bias)
      self.w_key= nn.Linear(d_in,d_out,bias= qkv_bias)
      self.w_value= nn.Linear(d_in,d_out,bias= qkv_bias)
      self.dropout= nn.Dropout(dropout)
      self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

  def forward(self,x):
    b,num_tokens,d_in= x.shape # New batch dimension b
    keys = self.w_key(x)
    query= self.w_query(x)
    value= self.w_value(x)
    print(keys.shape)

    atten_score= query @ keys.transpose(1,2)
    atten_score.masked_fill(self.mask.bool()[:num_tokens,:num_tokens],-torch.inf)
    atten_weight= torch.softmax(atten_score/keys.shape[-1]**0.5 ,dim=-1)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
    atten_weight=self.dropout(atten_weight)
    print(atten_weight.shape)
    print(value.shape)
    contex_vec= atten_weight @ value
    return contex_vec



In [ ]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)

torch.Size([2, 6, 3])
torch.Size([2, 6, 2])
torch.Size([2, 6, 6])
torch.Size([2, 6, 2])
context_vecs.shape: torch.Size([2, 6, 2])


In [ ]:
ca(batch)

torch.Size([2, 6, 2])
torch.Size([2, 6, 6])
torch.Size([2, 6, 2])


tensor([[[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
         [-0.5311, -0.1066],
         [-0.5299, -0.1081]],

        [[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
         [-0.5311, -0.1066],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)

In [ ]:
print(context_vecs)

tensor([[[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
         [-0.5311, -0.1066],
         [-0.5299, -0.1081]],

        [[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
         [-0.5311, -0.1066],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)


In [ ]:
class Multiheadattentionwrapper(nn.Module):
  def __init__(self,d_in,d_out,contex_length,dropout,num_heads,qkv_bias=False):
    super().__init__()
    self.heads=nn.ModuleList([CausalAttention(d_in,d_out,contex_length,dropout,qkv_bias) for _ in range(num_heads)])

  def forward(self,x):
    return torch.cat([head(x) for head in self.heads],dim=-1)


In [ ]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


It gives you all the machinery for parameters, buffers, and layers to be registered automatically.

Provides built-in methods like .parameters(), .to(device), .train(), .eval(), etc.

Ensures your sub-layers (like Linear, Dropout, etc.) are tracked and updated when training.

Without inheriting nn.Module, PyTorch wouldn’t know which tensors are trainable parameters.

🔹 2. What is nn.ModuleList?

nn.ModuleList is a container for holding a list of layers (modules).

It’s just like a Python list, but with one big difference:
PyTorch registers everything inside it as submodules of your parent module.

That means:

Their parameters will appear in .parameters().

They’ll automatically move to GPU/CPU when you call .to(device).

They’ll toggle correctly with .train() and .eval().

👉 If you used a normal Python list, PyTorch wouldn’t track those submodules, and their parameters wouldn’t update during training.

In [ ]:
torch.manual_seed(123)
context_length = batch.shape[1] # This is the number of tokens = 6
d_in, d_out = 3, 2
mha = Multiheadattentionwrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

torch.Size([2, 6, 2])
torch.Size([2, 6, 6])
torch.Size([2, 6, 2])
torch.Size([2, 6, 2])
torch.Size([2, 6, 6])
torch.Size([2, 6, 2])
tensor([[[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


Now wea are going to create a Multi-head attention in single class


In [ ]:
class MultiheadAttention(nn.module):
  def __init__(self,d_in,d_out,contex_length,dropout,num_heads,qkv_bias=False):
    super().__init__()
    assert(d_out % num_heads ==0), "d_out must be divisible by num_heads"
    self.d_out=d_out
    self.num_heads= num_heads
    self.head_dim= d_out // num_heads

    self.w_query=nn.Linear(d_in,d_out,qkv_bias)
    self.w_key=nn.Linear(d_in,d_out,qkv_bias)
    self.w_query=nn.Linear(d_in,d_out,qkv_bias)
    self.out_proj= nn.Linear(d_out,d_out)  # Linear layer to combine head outputs
    self.dropout= nn.Dropout(dropout)
    self.register_buffer("mask",torch.triu(torch.ones(contex_length,context_length),diagonal=1))

  def forward(self,x):
    b,num_tokens,d_in = x.shape
    keys =self.w_key(x) # Shape: (b, num_tokens, d_out)
    query= self.w_query(x)
    value= self.w_value(x)

    # We implicitly split the matrix by adding a `num_heads` dimension
    # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim).
    keys= keys.view(b,num_token,num_heads,head_dim)
    query=keys.view(b,num_token,num_heads,head_dim)
    value=keys.view(b,num_token,num_heads,head_dim)

    # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
    keys= keys.transpose(1,2)
    query=query.transpose(1,2)
    value=value.transpose(1,2)

    # Compute scaled dot-product attention (aka self-attention) with a causal mask
    atten_score= query @ keys.transpose(1,2)

    # Original mask truncated to the number of tokens and converted to boolean
    mask_bool = self.mask.bool()[:num_token,:num_token]

    # Use the mask to fill attention scores
    atten_score.masked_fill(mask_bool,-torch.inf)
    atten_weight=torch.softmax(atten_score / keys.shape[-1]**0.5 ,dim=-1)

    atten_weight= self.dropout(atten_weight)

    # Shape: (b, num_tokens, num_heads, head_dim)
    contex_vector= (atten_weight @ value).transpose(1,2)

    # Combine heads, where self.d_out = self.num_heads * self.head_dim
    context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
    context_vec = self.out_proj(context_vec) # optional projection

    return context_vec

1. What does .contiguous() mean?

In PyTorch, tensors are stored in memory as 1D arrays, with some stride info telling how to interpret them as multi-dimensional.
When you do operations like .transpose() or .view(), sometimes the tensor in memory is no longer contiguous (not stored in a simple sequential block).

👉 .contiguous() makes a new copy of the tensor in memory so that all elements are laid out sequentially again.

2. Why is this important before .view()?

.view() doesn’t actually rearrange values, it just reinterprets memory layout with a new shape.
If the tensor is non-contiguous, .view() might give wrong results or raise an error.

<h3><b>
Why do we care?
</b><br><u>
Some operations (like .view()) assume contiguous memory.

If the tensor is not contiguous, they fail or give wrong results.

.contiguous() makes a new copy in memory, arranging numbers as they appear, so you can safely reshape.
</u></h3>

<div class="alert alert-block alert-info">

Step 1: Reduce the projection dim to match desired output dim

Step 2: Use a Linear layer to combine head outputs

Step 3: Tensor shape: (b, num_tokens, d_out)

Step 4: We implicitly split the matrix by adding a `num_heads` dimension. Then we unroll last dim: (b,
num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)

Step 5: Transpose from shape (b, num_tokens, num_heads, head_dim) to (b, num_heads, num_tokens, head_dim)

Step 6: Compute dot product for each head

Step 7: Mask truncated to the number of tokens

Step 8: Use the mask to fill attention scores

Step 9: Tensor shape: (b, num_tokens, n_heads, head_dim)

Step 10: Combine heads, where self.d_out = self.num_heads * self.head_dim

Step 11: Add an optional linear projection
</div>